In [48]:
import ete3
import numpy as np
from scipy import linalg
import Sequence_Analysis_Routines as sar
import math

In [23]:
project_dir = 'D:/Project_Data/Project_3'
output_dir = project_dir + '/Output/Close_Species'
outgroup_species = 'NC_008596.1'

In [24]:
master_tree = ete3.Tree(output_dir + '/Trees/Concatenated_JC_Tree.treefile')
#print(master_tree)
outgroup = master_tree.search_nodes(name= outgroup_species)[0]
outgroup.delete()
#master_tree.write(format=0, outfile= output_dir + '/Trees/test.treefile')  
master_tree = ete3.Tree("(F:2.5,((D:0.5,E:0.5):2.0,((A:1,B:1):0.5, C:1.5):1):0.5);" )
print(master_tree)


   /-F
  |
--|      /-D
  |   /-|
  |  |   \-E
   \-|
     |      /-A
     |   /-|
      \-|   \-B
        |
         \-C


In [49]:
def transition_prob(state_1, state_2, transition_matrix, branch_length):
    #return linalg.expm(branch_length * transition_matrix)[state_1, state_2]
    temp = math.exp(-4.0*branch_length/3.0)
    if state_1 == state_2:
        ans = 0.25 - 0.25 * temp
    else:
        ans = 0.25 * 0.75 * temp
    return ans

In [26]:
num_states = 3
prior_probabilities = [1/3, 1/3, 1/3]
initial_states = {"A": 0, "B": 1, "C": 0, "D": 2, "E": 2, "F": 1}
prob_matrix = np.array([[-2,1,1],[1,-2,1],[1,1,-2]])
nodes_under_consideration = []
info_dict = {}
num_nodes = 0
for node in master_tree.traverse():
    num_nodes+=1
    if node.is_leaf():
        nodes_under_consideration.append(node)
        temp_probs = []
        for s in range(num_states):
            if initial_states[node.name] == s:
                temp_probs.append(1)
            else:
                temp_probs.append(0)
        info_dict[node] = temp_probs
while(len(nodes_under_consideration) < num_nodes):
    for n in nodes_under_consideration:
        if n.up in info_dict:
            continue
        sibling_group = [n]
        for p in n.get_sisters():
            sibling_group.append(p)
        num_not_in_dict = 0
        for x in sibling_group:
            if not(x in info_dict):
                num_not_in_dict +=1
        if num_not_in_dict == 0:
            new_probs = []
            for s in range(num_states):
                temp_prob = 1
                for x in sibling_group:
                    branch_length = x.dist
                    probs = info_dict[x]
                    temp_prob_2 = 0
                    for t in range(num_states):
                        temp_prob_2 += transition_prob(s, t, prob_matrix, branch_length)*probs[t]
                    temp_prob = temp_prob * temp_prob_2
                new_probs.append(temp_prob)
            info_dict[n.up] = new_probs
    nodes_under_consideration = list(info_dict.keys()) 
for node in master_tree.traverse():
    if node.is_root():
        ans = 0
        probs = info_dict[node]
        for s in range(num_states):
            ans = ans + prior_probabilities[s] * probs[s] 
#print(info_dict)
print(ans)

0.001504767335103887


In [45]:
state_list = [['A','B','C','D','E','F'],[0,1,0,2,2,1]]
substitution_matrix = np.array([[-2,1,1],[1,-2,1],[1,1,-2]])
sar.felsenstein_probability (state_list, 3, substitution_matrix, master_tree, 1)

0.001504767335103887

In [50]:
for i in range(90000):
    a = transition_prob(1,1,prob_matrix,1)